# Multi Step Forecasting

Import relevant modules

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pywt
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
keras.backend.clear_session()

Read coefficient, in this example a3 from DWT

In [ ]:
data_all = pd.read_csv(r'C:\Users\AliMS\Documents\project\code\data\10_2018_dwt_a3.csv', usecols = ['a3'])

Normalise data between 1 and 0

In [ ]:
scaler = MinMaxScaler()
data_all = scaler.fit_transform(data_all)
steps_ahead = 6

Create 10D targets

In [ ]:
Y_data_all = np.empty((data_all.shape[0] - steps_ahead, 1, steps_ahead))
length = data_all.shape[0] - steps_ahead
z = int
z = 0
for one_step in range (1, length):
    for multi_step in range (1, steps_ahead + 1):
        Y_data_all[one_step - 1, :, multi_step - 1] = data_all[one_step + z, 0]
        z = z + 1
    z = 0

Remove last 10 values from inputs, as the predictions can't be compared to a measured value

In [ ]:
X_data_all = data_all[: - steps_ahead]

Divide into test, validate and train data

In [ ]:
train_size = int(data_all.shape[0]*0.9)
X_data_train = X_data_all[:train_size, :]
X_data_test = X_data_all[train_size:, :]

In [ ]:
Y_data_train = Y_data_all[:train_size, :]
Y_data_test = Y_data_all[train_size:, :]

Create lag between targets and inputs

In [ ]:
X_train, Y_train = X_data_train, Y_data_train
X_test, Y_test = X_data_test, Y_data_test
X_all = data_all[: - steps_ahead, :]

Reshape arrays to 3D

In [ ]:
X_train = X_train.reshape(-1, 1, 1)
X_test = X_test.reshape(-1, 1, 1)
X_all = X_all.reshape(-1, 1, 1)

In [ ]:
X_train.shape, X_test.shape

Create Model, in this case LSTM. Replace with SimpleRNN or GRU.

In [ ]:
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences = True, input_shape = [None, 1]),
    keras.layers.LSTM(320, return_sequences = True),
    keras.layers.Dense(6, activation='sigmoid'),
])

Select optimizer and compile model

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=0.01)
model.compile(loss = 'mse', optimizer = optimizer)

Train model

In [ ]:
model.fit(X_train, Y_train, epochs = 20, validation_split = 0.1, verbose=1)

In [ ]:
score = model.evaluate(X_test, Y_test)
print('Score: {}'.format(score))

Predict Data

In [ ]:
Y_pred = model.predict(X_all)
Y_pred.shape

Select which step ahead

In [ ]:
Y_pred_data = np.empty((Y_pred.shape[0], 1))
eval_step = 1
for one_step in range (1, length + 1):
        Y_pred_data[one_step - 1, 0] = Y_pred[one_step - 1, 0, eval_step - 1]

Correct length

In [ ]:
Y_pred_data = scaler.inverse_transform(Y_pred_data)
end = steps_ahead - eval_step
if end > 0:
    Y_all = data_all[eval_step: - end,:]
else:
    Y_all = data_all[eval_step:,]
Y_all.shape

Denormalise Data

In [ ]:
Y_all = scaler.inverse_transform(Y_all)

Convert to pandas dataframe and rename column

In [ ]:
Y_pred_data = pd.DataFrame(Y_pred_data)
Y_pred_data = Y_pred_data.rename(columns = {0:'a3'})

Correct length at start of data

In [ ]:
extra_start = pd.DataFrame(index = range(eval_step), columns = range(1))
extra_start = extra_start.rename(columns = {0:'a3'})
extra_start.values[:] = Y_pred_data.at[0,'a3']
Y_pred_data = extra_start.append(Y_pred_data, ignore_index = True)
Y_pred_data.shape

Convert to pandas dataframe and rename column

In [ ]:
data_all_pd = pd.DataFrame(data_all)
data_all_pd = data_all_pd.rename(columns = {0:'a3'})
data_all_pd.shape

Correct length at end of data

In [ ]:
length_end = data_all_pd.shape[0] - steps_ahead
if end > 0:
    extra_end = pd.DataFrame(index = range(end), columns = range(1))
    extra_end = extra_end.rename(columns = {0:'a3'})
    for row in range (1, end + 1):
        extra_end.at[row - 1,'a3'] = data_all_pd.at[length_end + row - 1,'a3']
    Y_pred_data = Y_pred_data.append(extra_end, ignore_index = True)
Y_pred_data.shape

Write predicted data to csv file

In [ ]:
Y_pred_data.to_csv(r'C:\Users\AliMS\Documents\project\code\data\10_2018_pred_dwt_a3.csv')